In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nb_samples = 100
features = torch.randn(nb_samples, 10)
labels = torch.empty(nb_samples, dtype=torch.long)
adjacency = torch.randn(nb_samples, 5)
laplacian = torch.randn(nb_samples, 7)

dataset = TensorDataset(features, labels, adjacency, laplacian)
loader = DataLoader(
    dataset,
    batch_size=2
)

for batch_idx, (x, y, a, l) in enumerate(loader):
    print(x.shape, y.shape, a.shape, l.shape)

torch.Size([2, 10]) torch.Size([2]) torch.Size([2, 5]) torch.Size([2, 7])
torch.Size([2, 10]) torch.Size([2]) torch.Size([2, 5]) torch.Size([2, 7])
torch.Size([2, 10]) torch.Size([2]) torch.Size([2, 5]) torch.Size([2, 7])
torch.Size([2, 10]) torch.Size([2]) torch.Size([2, 5]) torch.Size([2, 7])
torch.Size([2, 10]) torch.Size([2]) torch.Size([2, 5]) torch.Size([2, 7])
torch.Size([2, 10]) torch.Size([2]) torch.Size([2, 5]) torch.Size([2, 7])
torch.Size([2, 10]) torch.Size([2]) torch.Size([2, 5]) torch.Size([2, 7])
torch.Size([2, 10]) torch.Size([2]) torch.Size([2, 5]) torch.Size([2, 7])
torch.Size([2, 10]) torch.Size([2]) torch.Size([2, 5]) torch.Size([2, 7])
torch.Size([2, 10]) torch.Size([2]) torch.Size([2, 5]) torch.Size([2, 7])
torch.Size([2, 10]) torch.Size([2]) torch.Size([2, 5]) torch.Size([2, 7])
torch.Size([2, 10]) torch.Size([2]) torch.Size([2, 5]) torch.Size([2, 7])
torch.Size([2, 10]) torch.Size([2]) torch.Size([2, 5]) torch.Size([2, 7])
torch.Size([2, 10]) torch.Size([2]) to

# GAN-BERT (in Pytorch and compatible with HuggingFace)

This is a Pytorch (+ **Huggingface** transformers) implementation of the GAN-BERT model from https://github.com/crux82/ganbert. While the original GAN-BERT was an extension of BERT, this implementation can be adapted to several architectures, ranging from Roberta to Albert!

**NOTE**: given that this implementation is different from the original one in Tensorflow, some results can be slighty different.


In [2]:
!pip install onnxruntime
!pip install onnx

     |████████████████████████████████| 4.9 MB 4.4 MB/s 
     |████████████████████████████████| 12.7 MB 4.3 MB/s 


Let's GO!

Required Imports.

In [3]:
!pip install transformers==4.3.2
import pandas as pd
import torch
import io
import torch.nn.functional as F
import random
import numpy as np
import time
import math
import datetime
import torch.nn as nn
from transformers import *
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#!pip install sentencepiece

##Set random values
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(seed_val)

     |████████████████████████████████| 1.8 MB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 31.4 MB/s 
     |████████████████████████████████| 895 kB 44.9 MB/s 


In [5]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


### Input Parameters


In [4]:
#--------------------------------
#  Transformer parameters
#--------------------------------
max_seq_length = 64
batch_size = 64

#--------------------------------
#  GAN-BERT specific parameters
#--------------------------------
# number of hidden layers in the generator, 
# each of the size of the output space
num_hidden_layers_g = 1; 
# number of hidden layers in the discriminator, 
# each of the size of the input space
num_hidden_layers_d = 1; 
# size of the generator's input noisy vectors
noise_size = 100
# dropout to be applied to discriminator's input vectors
out_dropout_rate = 0.2

# Replicate labeled data to balance poorly represented datasets, 
# e.g., less than 1% of labeled material
apply_balance = True

#--------------------------------
#  Optimization parameters
#--------------------------------
learning_rate_discriminator = 5e-5
learning_rate_generator = 5e-5
epsilon = 1e-8
num_train_epochs = 8
multi_gpu = False
# Scheduler
apply_scheduler = False
warmup_proportion = 0.1
# Print
print_each_n_step = 10

#--------------------------------
#  Adopted Tranformer model
#--------------------------------
# Since this version is compatible with Huggingface transformers, you can uncomment
# (or add) transformer models compatible with GAN

model_name = "bert-base-cased"
#model_name = "bert-base-uncased"
#model_name = "roberta-base"
#model_name = "albert-base-v2"
#model_name = "xlm-roberta-base"
#model_name = "amazon/bort"

#--------------------------------
#  Retrieve the TREC QC Dataset
#--------------------------------
! git clone https://github.com/crux82/ganbert

#  NOTE: in this setting 50 classes are involved
labeled_file = "./ganbert/data/labeled.tsv"
unlabeled_file = "./ganbert/data/unlabeled.tsv"
test_filename = "./ganbert/data/test.tsv"

# label_list = ["UNK_UNK","ABBR_abb", "ABBR_exp", "DESC_def", "DESC_desc", 
#               "DESC_manner", "DESC_reason", "ENTY_animal", "ENTY_body", 
#               "ENTY_color", "ENTY_cremat", "ENTY_currency", "ENTY_dismed", 
#               "ENTY_event", "ENTY_food", "ENTY_instru", "ENTY_lang", 
#               "ENTY_letter", "ENTY_other", "ENTY_plant", "ENTY_product", 
#               "ENTY_religion", "ENTY_sport", "ENTY_substance", "ENTY_symbol", 
#               "ENTY_techmeth", "ENTY_termeq", "ENTY_veh", "ENTY_word", "HUM_desc", 
#               "HUM_gr", "HUM_ind", "HUM_title", "LOC_city", "LOC_country", 
#               "LOC_mount", "LOC_other", "LOC_state", "NUM_code", "NUM_count", 
#               "NUM_date", "NUM_dist", "NUM_money", "NUM_ord", "NUM_other", 
#               "NUM_perc", "NUM_period", "NUM_speed", "NUM_temp", "NUM_volsize", 
#               "NUM_weight"]



Cloning into 'ganbert'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 77 (delta 33), reused 54 (delta 18), pack-reused 0
Unpacking objects: 100% (77/77), done.


Copy my custom train and test data from google drive to working directory

In [ ]:
import os
# os.getcwd()

!cp gdrive/MyDrive/Colab\ Notebooks/Corona_NLP_test.csv ganbert/data/
!cp gdrive/MyDrive/Colab\ Notebooks/Corona_NLP_train.csv ganbert/data/




cp: cannot stat 'gdrive/MyDrive/Colab Notebooks/Corona_NLP_test.csv': No such file or directory
cp: cannot stat 'gdrive/MyDrive/Colab Notebooks/Corona_NLP_train.csv': No such file or directory


In [5]:
df_train = pd.read_csv('/content/drive/MyDrive/Corona_NLP_train.csv', encoding='latin-1')

df_test = pd.read_csv('/content/drive/MyDrive/Corona_NLP_test.csv', encoding='latin-1')

df_train.head()
print(df_train.Sentiment.unique())
df_train = df_train.sample(frac=0.1)  
df_train.shape

['Neutral' 'Positive' 'Extremely Negative' 'Negative' 'Extremely Positive']


(4116, 6)

In [8]:
df_test.shape

(3798, 6)

In [ ]:
25/100*3798

949.5

In [9]:
label_list = ["UNK", "Neutral", "Positive", "Extremely Negative", "Negative", "Extremely Positive"]

In [10]:
df_train_for_ganbert = df_train.sample(frac=0.01) # use 1% of the labeled data for training
df_unlabeled = df_train.drop(df_train_for_ganbert.index)

print(df_train_for_ganbert.shape, df_unlabeled.shape)

(41, 6) (4075, 6)


In [11]:
# Change each of the sentiment for unlabeled data to Unknown
for i in df_unlabeled.index:
    df_unlabeled.at[i, "Sentiment"] = 'UNK'
    

In [12]:
# Defining function to format the dataset to be used in the dataloader
def get_examples(df):

    examples = []

    # iterate through each row
    for index, row in df.iterrows():
        examples.append((row['OriginalTweet'], row['Sentiment']))

    return examples

In [13]:
labeled_examples = get_examples(df_train_for_ganbert)
unlabeled_examples = get_examples(df_unlabeled)
test_examples = get_examples(df_test)

Load the Tranformer Model

In [14]:
transformer = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [15]:
# torch.save(transformer, 'transformer')
# transformer = torch.load('transformer')
# transformer.eval()

Function required to load the dataset

In [16]:
# def get_qc_examples(input_file):
#   """Creates examples for the training and dev sets."""
#   examples = []

#   with open(input_file, 'r') as f:
#       contents = f.read()
#       file_as_list = contents.splitlines()
#       for line in file_as_list[1:]:
#           split = line.split(" ")
#           question = ' '.join(split[1:])

#           text_a = question
#           inn_split = split[0].split(":")
#           label = inn_split[0] + "_" + inn_split[1]
#           examples.append((text_a, label))
#       f.close()

#   return examples

**Load** the input QC dataset (fine-grained)

In [17]:
#Load the examples
# labeled_examples = get_qc_examples(labeled_file)
# unlabeled_examples = get_qc_examples(unlabeled_file)
# test_examples = get_qc_examples(test_filename)

Functions required to convert examples into Dataloader

In [18]:
def generate_data_loader(input_examples, label_masks, label_map, do_shuffle = False, balance_label_examples = False):
  '''
  Generate a Dataloader given the input examples, eventually masked if they are 
  to be considered NOT labeled.
  '''
  examples = []

  # Count the percentage of labeled examples  
  num_labeled_examples = 0
  for label_mask in label_masks:
    if label_mask: 
      num_labeled_examples += 1
  label_mask_rate = num_labeled_examples/len(input_examples)

  # if required it applies the balance
  for index, ex in enumerate(input_examples): 
    if label_mask_rate == 1 or not balance_label_examples:
      examples.append((ex, label_masks[index]))
    else:
      # IT SIMULATE A LABELED EXAMPLE
      if label_masks[index]:
        balance = int(1/label_mask_rate)
        balance = int(math.log(balance,2))
        if balance < 1:
          balance = 1
        for b in range(0, int(balance)):
          examples.append((ex, label_masks[index]))
      else:
        examples.append((ex, label_masks[index]))
  
  #-----------------------------------------------
  # Generate input examples to the Transformer
  #-----------------------------------------------
  input_ids = []
  input_mask_array = []
  label_mask_array = []
  label_id_array = []

  # Tokenization 
  for (text, label_mask) in examples:
    encoded_sent = tokenizer.encode(text[0], add_special_tokens=True, max_length=max_seq_length, padding="max_length", truncation=True)
    input_ids.append(encoded_sent)
    label_id_array.append(label_map[text[1]])
    label_mask_array.append(label_mask)
  
  # Attention to token (to ignore padded input wordpieces)
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]                          
    input_mask_array.append(att_mask)
  # Convertion to Tensor
  input_ids = torch.tensor(input_ids) 
  input_mask_array = torch.tensor(input_mask_array)
  label_id_array = torch.tensor(label_id_array, dtype=torch.long)
  label_mask_array = torch.tensor(label_mask_array)

  # Building the TensorDataset
  dataset = TensorDataset(input_ids, input_mask_array, label_id_array, label_mask_array)

  if do_shuffle:
    sampler = RandomSampler
  else:
    sampler = SequentialSampler

  # Building the DataLoader
  return DataLoader(
              dataset,  # The training samples.
              sampler = sampler(dataset), 
              batch_size = batch_size, # Trains with this batch size.
              drop_last=True) 

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

Convert the input examples into DataLoader

In [19]:
label_map = {}
for (i, label) in enumerate(label_list):
  label_map[label] = i
#------------------------------
#   Load the train dataset
#------------------------------
train_examples = labeled_examples
#The labeled (train) dataset is assigned with a mask set to True
train_label_masks = np.ones(len(labeled_examples), dtype=bool)
#If unlabel examples are available
if unlabeled_examples:
  train_examples = train_examples + unlabeled_examples
  #The unlabeled (train) dataset is assigned with a mask set to False
  tmp_masks = np.zeros(len(unlabeled_examples), dtype=bool)
  train_label_masks = np.concatenate([train_label_masks,tmp_masks])

train_dataloader = generate_data_loader(train_examples, train_label_masks, label_map, do_shuffle = True, balance_label_examples = apply_balance)

#------------------------------
#   Load the test dataset
#------------------------------
#The labeled (test) dataset is assigned with a mask set to True
test_label_masks = np.ones(len(test_examples), dtype=bool)

test_dataloader = generate_data_loader(test_examples, test_label_masks, label_map, do_shuffle = False, balance_label_examples = False)

We define the Generator and Discriminator as discussed in https://www.aclweb.org/anthology/2020.acl-main.191/

In [7]:
#------------------------------
#   The Generator as in 
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Generator(nn.Module):
    def __init__(self, noise_size=100, output_size=512, hidden_sizes=[512], dropout_rate=0.1):
        super(Generator, self).__init__()
        layers = []
        hidden_sizes = [noise_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        layers.append(nn.Linear(hidden_sizes[-1],output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, noise):
        output_rep = self.layers(noise)
        return output_rep

#------------------------------
#   The Discriminator
#   https://www.aclweb.org/anthology/2020.acl-main.191/
#   https://github.com/crux82/ganbert
#------------------------------
class Discriminator(nn.Module):
    def __init__(self, input_size=512, hidden_sizes=[512], num_labels=2, dropout_rate=0.1):
        super(Discriminator, self).__init__()
        self.input_dropout = nn.Dropout(p=dropout_rate)
        layers = []
        hidden_sizes = [input_size] + hidden_sizes
        for i in range(len(hidden_sizes)-1):
            layers.extend([nn.Linear(hidden_sizes[i], hidden_sizes[i+1]), nn.LeakyReLU(0.2, inplace=True), nn.Dropout(dropout_rate)])

        self.layers = nn.Sequential(*layers) #per il flatten
        self.logit = nn.Linear(hidden_sizes[-1],num_labels+1) # +1 for the probability of this sample being fake/real.
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, input_rep):
        input_rep = self.input_dropout(input_rep)
        last_rep = self.layers(input_rep)
        logits = self.logit(last_rep)
        probs = self.softmax(logits)
        return last_rep, logits, probs

We instantiate the Discriminator and Generator

In [21]:
# The config file is required to get the dimension of the vector produced by 
# the underlying transformer
config = AutoConfig.from_pretrained(model_name)
hidden_size = int(config.hidden_size)
# Define the number and width of hidden layers
hidden_levels_g = [hidden_size for i in range(0, num_hidden_layers_g)]
hidden_levels_d = [hidden_size for i in range(0, num_hidden_layers_d)]

#-------------------------------------------------
#   Instantiate the Generator and Discriminator
#-------------------------------------------------
generator = Generator(noise_size=noise_size, output_size=hidden_size, hidden_sizes=hidden_levels_g, dropout_rate=out_dropout_rate)
discriminator = Discriminator(input_size=hidden_size, hidden_sizes=hidden_levels_d, num_labels=len(label_list), dropout_rate=out_dropout_rate)

# Put everything in the GPU if available
if torch.cuda.is_available():    
  generator.cuda()
  discriminator.cuda()
  transformer.cuda()
  if multi_gpu:
    transformer = torch.nn.DataParallel(transformer)

# print(config)

Let's go with the training procedure

In [22]:
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

#models parameters
transformer_vars = [i for i in transformer.parameters()]
d_vars = transformer_vars + [v for v in discriminator.parameters()]
g_vars = [v for v in generator.parameters()]

#optimizer
dis_optimizer = torch.optim.AdamW(d_vars, lr=learning_rate_discriminator)
gen_optimizer = torch.optim.AdamW(g_vars, lr=learning_rate_generator) 

#scheduler
if apply_scheduler:
  num_train_examples = len(train_examples)
  num_train_steps = int(num_train_examples / batch_size * num_train_epochs)
  num_warmup_steps = int(num_train_steps * warmup_proportion)

  scheduler_d = get_constant_schedule_with_warmup(dis_optimizer, 
                                           num_warmup_steps = num_warmup_steps)
  scheduler_g = get_constant_schedule_with_warmup(gen_optimizer, 
                                           num_warmup_steps = num_warmup_steps)

# For each epoch...
for epoch_i in range(0, num_train_epochs):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, num_train_epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    tr_g_loss = 0
    tr_d_loss = 0

    # Put the model into training mode.
    transformer.train() 
    generator.train()
    discriminator.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every print_each_n_step batches.
        if step % print_each_n_step == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_label_mask = batch[3].to(device)
        
        # Encode real data in the Transformer
        model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
        hidden_states = model_outputs[-1]
        
        # Generate fake data that should have the same distribution of the ones
        # encoded by the transformer. 
        # First noisy input are used in input to the Generator
        noise = torch.zeros(b_input_ids.shape[0],noise_size, device=device).uniform_(0, 1)
        # Gnerate Fake data
        gen_rep = generator(noise)

        # Generate the output of the Discriminator for real and fake data.
        # First, we put together the output of the tranformer and the generator
        disciminator_input = torch.cat([hidden_states, gen_rep], dim=0)
        # Then, we select the output of the disciminator
        features, logits, probs = discriminator(disciminator_input)

        # Finally, we separate the discriminator's output for the real and fake
        # data
        features_list = torch.split(features, batch_size)
        D_real_features = features_list[0]
        D_fake_features = features_list[1]
        
        logits_list = torch.split(logits, batch_size)
        D_real_logits = logits_list[0]
        D_fake_logits = logits_list[1]
        
        probs_list = torch.split(probs, batch_size)
        D_real_probs = probs_list[0]
        D_fake_probs = probs_list[1]

        #---------------------------------
        #  LOSS evaluation
        #---------------------------------
        # Generator's LOSS estimation
        g_loss_d = -1 * torch.mean(torch.log(1 - D_fake_probs[:,-1] + epsilon))
        g_feat_reg = torch.mean(torch.pow(torch.mean(D_real_features, dim=0) - torch.mean(D_fake_features, dim=0), 2))
        g_loss = g_loss_d + g_feat_reg
  
        # Disciminator's LOSS estimation
        logits = D_real_logits[:,0:-1]
        log_probs = F.log_softmax(logits, dim=-1)
        # The discriminator provides an output for labeled and unlabeled real data
        # so the loss evaluated for unlabeled data is ignored (masked)
        label2one_hot = torch.nn.functional.one_hot(b_labels, len(label_list))
        per_example_loss = -torch.sum(label2one_hot * log_probs, dim=-1)
        per_example_loss = torch.masked_select(per_example_loss, b_label_mask.to(device))
        labeled_example_count = per_example_loss.type(torch.float32).numel()

        # It may be the case that a batch does not contain labeled examples, 
        # so the "supervised loss" in this case is not evaluated
        if labeled_example_count == 0:
          D_L_Supervised = 0
        else:
          D_L_Supervised = torch.div(torch.sum(per_example_loss.to(device)), labeled_example_count)
                 
        D_L_unsupervised1U = -1 * torch.mean(torch.log(1 - D_real_probs[:, -1] + epsilon))
        D_L_unsupervised2U = -1 * torch.mean(torch.log(D_fake_probs[:, -1] + epsilon))
        d_loss = D_L_Supervised + D_L_unsupervised1U + D_L_unsupervised2U

        #---------------------------------
        #  OPTIMIZATION
        #---------------------------------
        # Avoid gradient accumulation
        gen_optimizer.zero_grad()
        dis_optimizer.zero_grad()

        # Calculate weigth updates
        # retain_graph=True is required since the underlying graph will be deleted after backward
        g_loss.backward(retain_graph=True)
        d_loss.backward() 
        
        # Apply modifications
        gen_optimizer.step()
        dis_optimizer.step()
   
        # A detail log of the individual losses
        #print("{0:.4f}\t{1:.4f}\t{2:.4f}\t{3:.4f}\t{4:.4f}".
        #      format(D_L_Supervised, D_L_unsupervised1U, D_L_unsupervised2U,
        #             g_loss_d, g_feat_reg))

        # Save the losses to print them later
        tr_g_loss += g_loss.item()
        tr_d_loss += d_loss.item()

        # Update the learning rate with the scheduler
        if apply_scheduler:
          scheduler_d.step()
          scheduler_g.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss_g = tr_g_loss / len(train_dataloader)
    avg_train_loss_d = tr_d_loss / len(train_dataloader)             
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss generetor: {0:.3f}".format(avg_train_loss_g))
    print("  Average training loss discriminator: {0:.3f}".format(avg_train_loss_d))
    print("  Training epcoh took: {:}".format(training_time))

    print("Saving the models...............................")
    # Saving the model
    torch.save(transformer, 'transformer')
    torch.save(discriminator, 'discriminator')

        
    # ========================================
    #     TEST ON THE EVALUATION DATASET
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our test set.
    print("")
    print("Running Test...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    transformer.eval() #maybe redundant
    discriminator.eval()
    generator.eval()

    # Tracking variables 
    total_test_accuracy = 0
   
    total_test_loss = 0
    nb_test_steps = 0

    all_preds = []
    all_labels_ids = []

    #loss
    nll_loss = torch.nn.CrossEntropyLoss(ignore_index=-1)

    # Evaluate data for one epoch
    for batch in test_dataloader:
        
        # Unpack this training batch from our dataloader. 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        
            model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
            hidden_states = model_outputs[-1]
            _, logits, probs = discriminator(hidden_states)
            ###log_probs = F.log_softmax(probs[:,1:], dim=-1)
            filtered_logits = logits[:,0:-1]
            # Accumulate the test loss.
            total_test_loss += nll_loss(filtered_logits, b_labels)
            
        # Accumulate the predictions and the input labels
        _, preds = torch.max(filtered_logits, 1)
        all_preds += preds.detach().cpu()
        all_labels_ids += b_labels.detach().cpu()


    # Report the final accuracy for this validation run.
    all_preds = torch.stack(all_preds).numpy()
    all_labels_ids = torch.stack(all_labels_ids).numpy()
    test_accuracy = np.sum(all_preds == all_labels_ids) / len(all_preds)
    print("  Accuracy: {0:.3f}".format(test_accuracy))

    # Calculate the average loss over all of the batches.
    avg_test_loss = total_test_loss / len(test_dataloader)
    avg_test_loss = avg_test_loss.item()
    
    # Measure how long the validation run took.
    test_time = format_time(time.time() - t0)
    
    print("  Test Loss: {0:.3f}".format(avg_test_loss))
    print("  Test took: {:}".format(test_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss generator': avg_train_loss_g,
            'Training Loss discriminator': avg_train_loss_d,
            'Valid. Loss': avg_test_loss,
            'Valid. Accur.': test_accuracy,
            'Training Time': training_time,
            'Test Time': test_time
        }
    )


======== Epoch 1 / 8 ========
Training...
  Batch    10  of     67.    Elapsed: 0:00:21.
  Batch    20  of     67.    Elapsed: 0:00:41.
  Batch    30  of     67.    Elapsed: 0:01:02.
  Batch    40  of     67.    Elapsed: 0:01:22.
  Batch    50  of     67.    Elapsed: 0:01:42.
  Batch    60  of     67.    Elapsed: 0:02:03.

  Average training loss generetor: 0.613
  Average training loss discriminator: 2.448
  Training epcoh took: 0:02:17
Saving the models...............................

Running Test...
  Accuracy: 0.259
  Test Loss: 1.769
  Test took: 0:00:27

======== Epoch 2 / 8 ========
Training...
  Batch    10  of     67.    Elapsed: 0:00:20.
  Batch    20  of     67.    Elapsed: 0:00:41.
  Batch    30  of     67.    Elapsed: 0:01:01.
  Batch    40  of     67.    Elapsed: 0:01:21.
  Batch    50  of     67.    Elapsed: 0:01:42.
  Batch    60  of     67.    Elapsed: 0:02:02.

  Average training loss generetor: 0.743
  Average training loss discriminator: 1.262
  Training epcoh took

In [23]:
# Loading the saved model
transformer = torch.load('transformer')
discriminator = torch.load('discriminator')

#This block is created to show how to load saved model and run test.
#I have used our test data for prediction and evaluation. You are free to use any data. 

In [24]:
print("")
print("Running Test...")

t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
transformer.eval() #maybe redundant
discriminator.eval()

# Tracking variables 
total_test_accuracy = 0

total_test_loss = 0
nb_test_steps = 0

all_preds = []
all_labels_ids = []

#loss
nll_loss = torch.nn.CrossEntropyLoss(ignore_index=-1)

# Evaluate data for one epoch
for batch in test_dataloader:
    
    # Unpack this training batch from our dataloader. 
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        
        model_outputs = transformer(b_input_ids, attention_mask=b_input_mask)
        hidden_states = model_outputs[-1]
        _, logits, probs = discriminator(hidden_states)
        ###log_probs = F.log_softmax(probs[:,1:], dim=-1)
        filtered_logits = logits[:,0:-1]
        # Accumulate the test loss.
        total_test_loss += nll_loss(filtered_logits, b_labels)
        
    # Accumulate the predictions and the input labels
    _, preds = torch.max(filtered_logits, 1)
    all_preds += preds.detach().cpu()
    all_labels_ids += b_labels.detach().cpu()

# Report the final accuracy for this validation run.
all_preds = torch.stack(all_preds).numpy()
all_labels_ids = torch.stack(all_labels_ids).numpy()
test_accuracy = np.sum(all_preds == all_labels_ids) / len(all_preds)
print("  Accuracy: {0:.3f}".format(test_accuracy))

# Calculate the average loss over all of the batches.
avg_test_loss = total_test_loss / len(test_dataloader)
avg_test_loss = avg_test_loss.item()

# Measure how long the validation run took.
test_time = format_time(time.time() - t0)
  
print("  Test Loss: {0:.3f}".format(avg_test_loss))
print("  Test took: {:}".format(test_time))



Running Test...
  Accuracy: 0.246
  Test Loss: 4.728
  Test took: 0:00:28


In [ ]:
# See the predictions
all_preds

array([4, 4, 4, ..., 5, 4, 4])

In [ ]:
for stat in training_stats:
  print(stat)

print("\nTraining complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

{'epoch': 1, 'Training Loss generator': 0.6157938904726683, 'Training Loss discriminator': 2.457873319511983, 'Valid. Loss': 1.844170093536377, 'Valid. Accur.': 0.2576800847457627, 'Training Time': '0:02:43', 'Test Time': '0:00:33'}

Training complete!
Total training took 0:04:36 (h:mm:ss)


# Convert to onnx

In [8]:
transformer = torch.load('/content/drive/MyDrive/GANBERT2/transformer', map_location = "cpu")
discriminator = torch.load('/content/drive/MyDrive/GANBERT2/discriminator', map_location = "cpu")

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [10]:
transformer.eval()
discriminator.eval()

Discriminator(
  (input_dropout): Dropout(p=0.2, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=768, out_features=768, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout(p=0.2, inplace=False)
  )
  (logit): Linear(in_features=768, out_features=7, bias=True)
  (softmax): Softmax(dim=-1)
)

In [11]:
encoded_dict = tokenizer.encode_plus(
                    "ahmed mohamed kotb",                     
                    add_special_tokens = True, 
                    max_length = 64,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    return_tensors = 'pt',     
                )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [12]:
def export_onnx_model(model, onnx_model_path):
    with torch.no_grad():
        inputs = {'input_ids':      torch.ones(1, 64, dtype=torch.int64),
            'attention_mask': torch.ones(1, 64, dtype=torch.int64)}
        outputs = model(encoded_dict["input_ids"], encoded_dict["attention_mask"])
        symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}
        torch.onnx.export(model,                                            # model being run
                    (inputs['input_ids'],                             # model input (or a tuple for multiple inputs)
                    inputs['attention_mask']),                                       # model input (or a tuple for multiple inputs)
                    onnx_model_path,                                # where to save the model (can be a file or file-like object)
                    opset_version=11,
                    verbose=True,                                 # the ONNX version to export the model to
                    do_constant_folding=True,                         # whether to execute constant folding for optimization
                    input_names=['input_ids',                         # the model's input names
                                'attention_mask'],
                    output_names=['output'],                    # the model's output names
                    dynamic_axes={'input_ids': symbolic_names,        # variable length axes
                                'attention_mask' : symbolic_names})
        print("ONNX Model exported to {0}".format(onnx_model_path))

export_onnx_model(transformer, "bert.onnx")

/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:1760: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  input_tensor.shape[chunk_dim] == tensor_shape for input_tensor in input_tensors


graph(%input_ids : Long(*, *, strides=[64, 1], requires_grad=0, device=cpu),
      %attention_mask : Long(*, *, strides=[64, 1], requires_grad=0, device=cpu),
      %embeddings.position_ids : Long(1, 512, strides=[512, 1], requires_grad=0, device=cpu),
      %embeddings.word_embeddings.weight : Float(28996, 768, strides=[768, 1], requires_grad=1, device=cpu),
      %embeddings.position_embeddings.weight : Float(512, 768, strides=[768, 1], requires_grad=1, device=cpu),
      %embeddings.token_type_embeddings.weight : Float(2, 768, strides=[768, 1], requires_grad=1, device=cpu),
      %embeddings.LayerNorm.weight : Float(768, strides=[1], requires_grad=1, device=cpu),
      %embeddings.LayerNorm.bias : Float(768, strides=[1], requires_grad=1, device=cpu),
      %encoder.layer.0.attention.self.query.bias : Float(768, strides=[1], requires_grad=1, device=cpu),
      %encoder.layer.0.attention.self.key.bias : Float(768, strides=[1], requires_grad=1, device=cpu),
      %encoder.layer.0.atten

In [13]:
from onnxruntime.quantization import quantize_dynamic, QuantType
import os
quantized_onnx_model_path = 'quantizebert1.onnx'
quantize_dynamic(
    '/content/bert.onnx',
    quantized_onnx_model_path,
    activation_type=QuantType.QUInt8,
    weight_type=QuantType.QUInt8
)
print('ONNX full precision model size (MB):', os.path.getsize('/content/bert.onnx') / (1024 * 1024))
print('ONNX quantized model size (MB):', os.path.getsize(quantized_onnx_model_path) / (1024 * 1024))

ONNX full precision model size (MB): 414.75461196899414
ONNX quantized model size (MB): 104.79691123962402
